In [1]:
import spikeinterface as si
import matplotlib.pyplot as plt
%matplotlib inline
import spikeinterface.extractors as se
import spikeinterface.preprocessing as spre
import spikeinterface.sorters as ss
import spikeinterface.widgets as sw
import spikeinterface.qualitymetrics as sqm
import json
import numpy as np
import pandas as pd
import seaborn as sns

/home/ubuntu/.conda/envs/spike_sorting_jct/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


- Dynamic

In [13]:
def get_trigger_time_range(file_path):
    recording = se.read_blackrock(file_path=file_path)
    recording = recording.channel_slice(channel_ids=['98'])
    trace_mat = recording.get_traces(start_frame= 10000,
                                    end_frame= 1350 * 10000,
                                    channel_ids= ['98'])
    trace_mat = trace_mat.astype(int)
    trace_mat[trace_mat < 20000] = 0
    trace_mat[trace_mat >= 20000] = 1

    starts = []
    ends = []

    for i in range(1, len(trace_mat)):
        if trace_mat[i] == 1 and trace_mat[i-1] == 0:
            starts.append(i+1)
        elif trace_mat[i] == 0 and trace_mat[i-1] == 1:
            ends.append(i)

    df = pd.DataFrame({'start': starts, 'end': ends})
    df = df[2:-2]
    df['end'] = df['start'] + 5000
    date = file_path.split("/")[8]
    date = date.split("_")[1]
    df['date'] = date
    return df


In [14]:
import os
recording_dir = os.listdir("/media/ubuntu/sda/data/mouse2/ns4/grating")
trigger_time = pd.DataFrame()
#recording_dir.remove("mouse6_102122_natural_image_001.ns4")

for recording in recording_dir:
    temp = get_trigger_time_range(f'/media/ubuntu/sda/data/mouse2/ns4/grating/{recording}')
    trigger_time = pd.concat((trigger_time, temp), axis = 0)

In [15]:
for date in trigger_time['date'].unique():
    temp = trigger_time[trigger_time['date'] == date]
    print(f'{date}: {len(temp)}')

042322: 180
052322: 180
022223: 180
062422: 180
031722: 180
082322: 180
102022: 180
112822: 180
122022: 180
072322: 180
092022: 180


###### Trigger order

In [16]:
def get_image_order(file_path):
    from scipy.io import loadmat
    mat_data = loadmat(file_path)
    mat_data = mat_data['Image_show_indices_dynamic']
    mat_data_pure = mat_data.ravel()

    row_indices = np.repeat(np.arange(mat_data.shape[0]), mat_data.shape[1])
    suffixes = np.array([f"_{i+1}" for i in row_indices])
    mat_data_order = np.array([f"{val}{suffix}" for val, suffix in zip(mat_data_pure, suffixes)])
    return mat_data_pure, mat_data_order

In [17]:
import os
trigger_dir = os.listdir("/media/ubuntu/sda/data/trigger/mouse2/")

In [24]:
date_order = ['031722', '042322', '052322', '062422', '072322', '082322', '092022', '102022', '112822', '122022', '022223']

In [25]:
image_order_dict = {}
for date in date_order:
    image_order_dict[date] = {}
    trigger = [s for s in trigger_dir if date in s][0]
    mat_dir = os.listdir(f"/media/ubuntu/sda/data/trigger/mouse2/{trigger}/")
    mat = [s for s in mat_dir if 'Dynamic' in s][0]
    image_order_dict[date]['pure'], image_order_dict[date]['order'] = get_image_order(f"/media/ubuntu/sda/data/trigger/mouse2/{trigger}/{mat}")

In [26]:
trigger_time['image'] = None
trigger_time['order'] = None
for date in trigger_time['date'].unique():
    trigger_time.loc[trigger_time['date'] == date, 'image'] = image_order_dict[date]['pure']
    trigger_time.loc[trigger_time['date'] == date, 'order'] = image_order_dict[date]['order']

In [27]:
trigger_time.to_csv('trigger_time_dynamic.tsv', sep = '\t')


- Stat

In [28]:
def get_trigger_time_range(file_path):
    recording = se.read_blackrock(file_path=file_path)
    recording = recording.channel_slice(channel_ids=['98'])
    trace_mat = recording.get_traces(start_frame= 1350 * 10000,
                                    end_frame= recording.get_duration() * 10000,
                                    channel_ids= ['98'])
    trace_mat = trace_mat.astype(int)
    trace_mat[trace_mat < 20000] = 0
    trace_mat[trace_mat >= 20000] = 1

    starts = []
    ends = []

    for i in range(1, len(trace_mat)):
        if trace_mat[i] == 1 and trace_mat[i-1] == 0:
            starts.append(i+1)
        elif trace_mat[i] == 0 and trace_mat[i-1] == 1:
            ends.append(i)

    df = pd.DataFrame({'start': starts, 'end': ends})
    df = df[2:-2]
    df['end'] = df['start'] + 5000
    date = file_path.split("/")[8]
    date = date.split("_")[1]
    df['date'] = date
    return df

In [29]:
import os
recording_dir = os.listdir("/media/ubuntu/sda/data/mouse2/ns4/grating")
trigger_time = pd.DataFrame()
#recording_dir.remove("mouse6_102122_natural_image_001.ns4")

for recording in recording_dir:
    temp = get_trigger_time_range(f'/media/ubuntu/sda/data/mouse2/ns4/grating/{recording}')
    trigger_time = pd.concat((trigger_time, temp), axis = 0)

In [31]:
for date in trigger_time['date'].unique():
    temp = trigger_time[trigger_time['date'] == date]
    print(f'{date}: {len(temp)}')

042322: 270
052322: 270
022223: 270
062422: 270
031722: 270
082322: 270
102022: 270
112822: 270
122022: 270
072322: 270
092022: 270


In [32]:
def get_image_order(file_path):
    from scipy.io import loadmat
    mat_data = loadmat(file_path)
    mat_data = mat_data['Image_show_indices_static']
    mat_data_pure = mat_data.ravel()

    row_indices = np.repeat(np.arange(mat_data.shape[0]), mat_data.shape[1])
    suffixes = np.array([f"_{i+1}" for i in row_indices])
    mat_data_order = np.array([f"{val}{suffix}" for val, suffix in zip(mat_data_pure, suffixes)])
    return mat_data_pure, mat_data_order

In [33]:
image_order_dict = {}
for date in date_order:
    image_order_dict[date] = {}
    trigger = [s for s in trigger_dir if date in s][0]
    mat_dir = os.listdir(f"/media/ubuntu/sda/data/trigger/mouse2/{trigger}/")
    mat = [s for s in mat_dir if 'Static' in s][0]
    image_order_dict[date]['pure'], image_order_dict[date]['order'] = get_image_order(f"/media/ubuntu/sda/data/trigger/mouse2/{trigger}/{mat}")

trigger_time['image'] = None
trigger_time['order'] = None
for date in trigger_time['date'].unique():
    trigger_time.loc[trigger_time['date'] == date, 'image'] = image_order_dict[date]['pure']
    trigger_time.loc[trigger_time['date'] == date, 'order'] = image_order_dict[date]['order']

trigger_time.to_csv('trigger_time_static.tsv', sep = '\t')